In [11]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from dash import dash, html, dcc , dash_table
from dash.dependencies import Input, Output
from plotly.offline import iplot , plot
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import ast # for converting string  to list 
from collections import Counter


In [12]:

# Load your dataset
df = pd.read_csv('dataset/Spotify-2000.csv')


In [13]:
user_data = pd.read_csv('dataset/spotify_archive/Spotify Quarterly.csv')

In [14]:
user_data.head()

,Date,Monthly Active User (MAU),Premium Users
0,Q1 2015,68,18
1,Q2 2015,77,22
2,Q3 2015,82,24
3,Q4 2015,91,28
4,Q1 2016,96,30


In [15]:
# Plot
fig = px.line(user_data, x='Date', y=['Monthly Active User (MAU)', 'Premium Users'], title='Spotify MAUs and Premium Users Over Quarters')
fig.update_layout(xaxis_title='Quarter', yaxis_title='Millions')
fig.show()

In [16]:
# Bar chart example
fig = px.bar(user_data, x='Date', y=['Monthly Active User (MAU)', 'Premium Users'], barmode='group', title='Spotify MAUs and Premium Users Over Quarters')
fig.update_layout(xaxis_title='Quarter', yaxis_title='Millions')
fig.show()

In [17]:
world_user = pd.read_csv('dataset/spotify_archive/World Users.csv')

In [18]:
world_user

,Year,Europe,North America,Latin America,Rest of World
0,2016,40,36,20,8
1,2017,51,46,28,13
2,2018,66,56,38,20
3,2019,83,65,49,35
4,2020,102,77,66,54
5,2021,124,88,80,73
6,2022,138,97,95,103
7,2023,159,111,116,165


In [19]:
# Creating a stacked area chart
fig = px.area(world_user, x='Year', y=['Europe', 'North America', 'Latin America', 'Rest of World'],
              title='Spotify Users by Region Over Years',
              labels={'value': 'Users (millions)', 'variable': 'Region'},
              facet_col_wrap=1)

# Update layout
fig.update_layout(xaxis_title='Year', yaxis_title='Number of Users (millions)')
fig.show()


In [21]:

spotify_2k = pd.read_csv('dataset/spotify_archive/Spotify-2000.csv')  # Make sure to adjust this to your dataset path


In [22]:
spotify_2k.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


In [23]:
spotify_2k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Index                   1994 non-null   int64 
 1   Title                   1994 non-null   object
 2   Artist                  1994 non-null   object
 3   Top Genre               1994 non-null   object
 4   Year                    1994 non-null   int64 
 5   Beats Per Minute (BPM)  1994 non-null   int64 
 6   Energy                  1994 non-null   int64 
 7   Danceability            1994 non-null   int64 
 8   Loudness (dB)           1994 non-null   int64 
 9   Liveness                1994 non-null   int64 
 10  Valence                 1994 non-null   int64 
 11  Length (Duration)       1994 non-null   object
 12  Acousticness            1994 non-null   int64 
 13  Speechiness             1994 non-null   int64 
 14  Popularity              1994 non-null   int64 
dtypes: i

In [24]:
numerical_stats = spotify_2k.describe()
numerical_stats.transpose()

,count,mean,std,min,25%,50%,75%,max
Index,1994.0,997.500000,575.762538,1.0,499.25,997.5,1495.75,1994.0
Year,1994.0,1992.992979,16.116048,1956.0,1979.00,1993.0,2007.00,2019.0
Beats Per Minute (BPM),1994.0,120.215647,28.028096,37.0,99.00,119.0,136.00,206.0
Energy,1994.0,59.679539,22.154322,3.0,42.00,61.0,78.00,100.0
Danceability,1994.0,53.238215,15.351507,10.0,43.00,53.0,64.00,96.0
Loudness (dB),1994.0,-9.008526,3.647876,-27.0,-11.00,-8.0,-6.00,-2.0
Liveness,1994.0,19.012036,16.727378,2.0,9.00,12.0,23.00,99.0
Valence,1994.0,49.408726,24.858212,3.0,29.00,47.0,69.75,99.0
Acousticness,1994.0,28.858074,29.011986,0.0,3.00,18.0,50.00,99.0
Speechiness,1994.0,4.994985,4.401566,2.0,3.00,4.0,5.00,55.0


In [25]:
# Let's find out the top 10 artists with the most tracks in the dataset
top_artists = spotify_2k['Artist'].value_counts().head(10)

top_artists

Artist
Queen                 37
The Beatles           36
Coldplay              27
U2                    26
The Rolling Stones    24
Bruce Springsteen     23
Michael Jackson       23
ABBA                  22
David Bowie           21
Fleetwood Mac         18
Name: count, dtype: int64

In [26]:
top_artists_df = top_artists.reset_index()
top_artists_df.columns = ['Artist', 'Count']

# Create a bar chart
fig = px.bar(top_artists_df, x='Count', y='Artist', text='Count', orientation='h',
             title="2000's Top 10 Artists by Count",
             labels={'Count': 'Count', 'Artist': 'Artist'},
             color='Count',  # Color the bars by the count
             color_continuous_scale='Agsunset')

# Improve layout
fig.update_layout(
    xaxis_title="Count",
    yaxis_title="",
    coloraxis_showscale=True,  # Hide the color scale
    yaxis={'categoryorder':'total ascending'},  # Ensure the highest values are on top
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.show()

In [27]:
characteristics = ['Danceability', 'Energy', 'Loudness (dB)', 'Speechiness', 'Acousticness', 'Popularity', 'Liveness', 'Valence', 'Beats Per Minute (BPM)']


In [28]:
# Calculate the correlation matrix for a subset of the numerical variables
correlation_matrix = df[characteristics].corr()

correlation_matrix

,Danceability,Energy,Loudness (dB),Speechiness,Acousticness,Popularity,Liveness,Valence,Beats Per Minute (BPM)
Danceability,1.000000,0.139616,0.044235,0.125229,-0.135769,0.144344,-0.103063,0.514564,-0.140602
Energy,0.139616,1.000000,0.735711,0.205865,-0.665156,0.103393,0.174118,0.405175,0.156644
Loudness (dB),0.044235,0.735711,1.000000,0.125090,-0.451635,0.165527,0.098257,0.147041,0.092927
Speechiness,0.125229,0.205865,0.125090,1.000000,-0.098256,0.111689,0.092594,0.107102,0.085598
Acousticness,-0.135769,-0.665156,-0.451635,-0.098256,1.000000,-0.087604,-0.046206,-0.239729,-0.122472
Popularity,0.144344,0.103393,0.165527,0.111689,-0.087604,1.000000,-0.111978,0.095911,-0.003181
Liveness,-0.103063,0.174118,0.098257,0.092594,-0.046206,-0.111978,1.000000,0.050667,0.016256
Valence,0.514564,0.405175,0.147041,0.107102,-0.239729,0.095911,0.050667,1.000000,0.059653
Beats Per Minute (BPM),-0.140602,0.156644,0.092927,0.085598,-0.122472,-0.003181,0.016256,0.059653,1.000000


In [29]:
# Generate text labels for the heatmap
text_labels = [[f'{val:.2f}' for val in row] for row in correlation_matrix.values]

# Create a heatmap with text labels inside each cell
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu_r',  # Change the colorscale to coolwarm
    zmid=0,  # Center the color scale at zero
    text=text_labels,
    texttemplate="%{text}",
    hoverinfo="text"
))

# Update layout for compact display
fig.update_layout(
    title='Correlation Matrix Heatmap',
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis=dict(tickmode='array', tickvals=np.arange(len(correlation_matrix.columns)), ticktext=correlation_matrix.columns),
    yaxis=dict(tickmode='array', tickvals=np.arange(len(correlation_matrix.columns)), ticktext=correlation_matrix.columns),
    autosize=True,
    margin=dict(l=10, r=10, b=10, t=50)  # Reduce margins to make the plot more compact
)

# Show the figure
fig.show()

In [30]:
spotify_2k23 = pd.read_csv('dataset/spotify_archive/spotify-2023.csv',encoding='latin1')

In [31]:
spotify_2k23.head()


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [32]:
spotify_2k23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [33]:
print(f"number of artists '{spotify_2k23['artist(s)_name'].nunique()}'")

number of artists '645'


In [34]:
artist_counts = spotify_2k23['artist(s)_name'].value_counts()
artist_counts

artist(s)_name
Taylor Swift                 34
The Weeknd                   22
Bad Bunny                    19
SZA                          19
Harry Styles                 17
                             ..
Karol G, Ovy On The Drums     1
Coolio, L.V.                  1
Kordhell                      1
Kenia OS                      1
Feid, Sech, Jhayco            1
Name: count, Length: 645, dtype: int64

In [35]:
colors = ["#8c0404","#f25ed0","#000000","#16A085","#34495E",
           "#21618C ","#512E5F","#45B39D","#AAB7B8 ","#20B2AA",
           "#FF69B4","#00CED1","#FF7F50","#7FFF00","#DA70D6"]

In [37]:
px.bar(artist_counts[:15],
             text_auto = True,
             color = artist_counts[:15].index,
             color_discrete_sequence = colors, 
             labels=dict(index="Count Names",value="Count"),
             title = '2023 Top 15 Artists with Most Songs'
            )
            # )

In [38]:
spotify_2k23.columns

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%'],
      dtype='object')

In [39]:
characteristics = ['in_spotify_playlists','danceability_%', 'energy_%', 'instrumentalness_%', 'speechiness_%', 'acousticness_%', 'liveness_%', 'valence_%', 'bpm']
correlation_matrix = spotify_2k23[characteristics].corr()


In [40]:

# Generate text labels for the heatmap
text_labels = [[f'{val:.2f}' for val in row] for row in correlation_matrix.values]

# Create a heatmap with text labels inside each cell
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu_r',  # Change the colorscale to coolwarm
    zmid=0,  # Center the color scale at zero
    text=text_labels,
    texttemplate="%{text}",
    hoverinfo="text"
))

# Update layout for compact display
fig.update_layout(
    title='Spotify 2023-Correlation Matrix Heatmap',
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis=dict(tickmode='array', tickvals=np.arange(len(correlation_matrix.columns)), ticktext=correlation_matrix.columns),
    yaxis=dict(tickmode='array', tickvals=np.arange(len(correlation_matrix.columns)), ticktext=correlation_matrix.columns),
    autosize=True,
    margin=dict(l=10, r=10, b=10, t=50)  # Reduce margins to make the plot more compact
)

# Show the figure
fig.show()


In [41]:
# Convert 'Length (Duration)' from string to integer
# Remove commas and convert to seconds
spotify_2k['Length (Duration)'] = spotify_2k['Length (Duration)'].str.replace(',', '')
spotify_2k['Length (Duration)'] = pd.to_timedelta(spotify_2k['Length (Duration)']).dt.total_seconds()

# Convert to integer
spotify_2k['Length (Duration)'] = spotify_2k['Length (Duration)'].astype(int)

In [42]:

# Selecting features for clustering
features = spotify_2k[['Beats Per Minute (BPM)', 'Energy', 'Danceability', 'Loudness (dB)', 'Valence', 'Acousticness', 'Speechiness']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [43]:

# Clustering
k = 5  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=0)
spotify_2k['Cluster'] = kmeans.fit_predict(features_scaled)


In [44]:
# Reduce dimensions
pca = PCA(2)
features_reduced = pca.fit_transform(features_scaled)

# Create a DataFrame for the plot
plot_df = pd.DataFrame(features_reduced, columns=['PC1', 'PC2'])
plot_df['Cluster'] = spotify_2k['Cluster']

# Plotting
fig = px.scatter(plot_df, x='PC1', y='PC2', color='Cluster', title='Music Data Clustering (2D PCA)',
                 color_continuous_scale=px.colors.qualitative.D3)
fig.show()


In [45]:
# Reduce dimensions to 3D
pca_3d = PCA(3)
features_3d = pca_3d.fit_transform(features_scaled)

# Create a DataFrame for the plot
plot_df_3d = pd.DataFrame(features_3d, columns=['PC1', 'PC2', 'PC3'])
plot_df_3d['Cluster'] = spotify_2k['Cluster']

# Plotting 3D
fig_3d = px.scatter_3d(plot_df_3d, x='PC1', y='PC2', z='PC3', color='Cluster', title='Music Data Clustering (3D PCA)',
                       color_continuous_scale=px.colors.qualitative.D3)
fig_3d.show()


In [46]:
artist_10k  =  pd.read_csv('dataset/spotify_archive/Top_10k_Artists.csv/Artists.csv')


In [47]:
artist_10k.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9488 entries, 0 to 9487
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        9488 non-null   object
 1   ID          9488 non-null   object
 2   Gender      7829 non-null   object
 3   Age         9488 non-null   int64 
 4   Country     6305 non-null   object
 5   Genres      9488 non-null   object
 6   Popularity  9488 non-null   int64 
 7   Followers   9488 non-null   int64 
 8   URI         9488 non-null   object
dtypes: int64(3), object(6)
memory usage: 667.3+ KB


In [48]:
artist_10k = artist_10k.drop(['ID', 'URI'], axis=1)

In [49]:
artist_10k['Country'].head()


0    CA
1    US
2    GB
3    CO
4    PR
Name: Country, dtype: object

In [50]:
# Print unique country names
print(artist_10k['Country'].unique())

['CA' 'US' 'GB' 'CO' 'PR' nan 'NO' 'FR' 'AU' 'PA' 'SE' 'DE' 'NL' 'BR' 'DK'
 'DO' 'AR' 'MX' 'JM' 'PH' 'KR' 'ES' 'AT' 'IT' 'VE' 'NZ' 'IE' 'TR' 'CU'
 'BE' 'RO' 'IL' 'IN' 'PL' 'JP' 'UY' 'FI' 'ID' 'DZ' 'XK' 'TW' 'SG' 'AG'
 'CL' 'PT' 'ZA' 'NG' 'HK' 'AW' 'AL' 'GR' 'SN' 'BA' 'BD' 'BO' 'LT' 'UA'
 'CN' 'RU' 'IS' 'HU' 'PK' 'CV' 'PE' 'MY' 'HN' 'LV' 'CH' 'BG' 'SK' 'CD'
 'TH' 'MK' 'EE' 'CM' 'VN' 'SI' 'GT' 'JO' 'CZ' 'MD' 'NI' 'HR' 'TN' 'ML'
 'GE' 'AO' 'KZ' 'XE' 'ST' 'GH' 'SR' 'BY' 'VG' 'SB' 'BF' 'XW' 'MA' 'CI'
 'SU' 'IR' 'YU' 'PY']


In [51]:
country_mapping = {
    'CA': 'Canada',
    'US': 'United States',
    'GB': 'United Kingdom',
    'CO': 'Colombia',
    'PR': 'Puerto Rico',
    'NO': 'Norway',
    'FR': 'France',
    'AU': 'Australia',
    'PA': 'Panama',
    'SE': 'Sweden',
    'DE': 'Germany',
    'NL': 'Netherlands',
    'BR': 'Brazil',
    'DK': 'Denmark',
    'DO': 'Dominican Republic',
    'AR': 'Argentina',
    'MX': 'Mexico',
    'JM': 'Jamaica',
    'PH': 'Philippines',
    'KR': 'South Korea',
    'ES': 'Spain',
    'AT': 'Austria',
    'IT': 'Italy',
    'VE': 'Venezuela',
    'NZ': 'New Zealand',
    'IE': 'Ireland',
    'TR': 'Turkey',
    'CU': 'Cuba',
    'BE': 'Belgium',
    'RO': 'Romania',
    'IL': 'Israel',
    'IN': 'India',
    'PL': 'Poland',
    'JP': 'Japan',
    'UY': 'Uruguay',
    'FI': 'Finland',
    'ID': 'Indonesia',
    'DZ': 'Algeria',
    'XK': 'Kosovo',
    'TW': 'Taiwan',
    'SG': 'Singapore',
    'AG': 'Antigua and Barbuda',
    'CL': 'Chile',
    'PT': 'Portugal',
    'ZA': 'South Africa',
    'NG': 'Nigeria',
    'HK': 'Hong Kong',
    'AW': 'Aruba',
    'AL': 'Albania',
    'GR': 'Greece',
    'SN': 'Senegal',
    'BA': 'Bosnia and Herzegovina',
    'BD': 'Bangladesh',
    'BO': 'Bolivia',
    'LT': 'Lithuania',
    'UA': 'Ukraine',
    'CN': 'China',
    'RU': 'Russia',
    'IS': 'Iceland',
    'HU': 'Hungary',
    'PK': 'Pakistan',
    'CV': 'Cape Verde',
    'PE': 'Peru',
    'MY': 'Malaysia',
    'HN': 'Honduras',
    'LV': 'Latvia',
    'CH': 'Switzerland',
    'BG': 'Bulgaria',
    'SK': 'Slovakia',
    'CD': 'Democratic Republic of the Congo',
    'TH': 'Thailand',
    'MK': 'North Macedonia',
    'EE': 'Estonia',
    'CM': 'Cameroon',
    'VN': 'Vietnam',
    'SI': 'Slovenia',
    'GT': 'Guatemala',
    'JO': 'Jordan',
    'CZ': 'Czech Republic',
    'MD': 'Moldova',
    'NI': 'Nicaragua',
    'HR': 'Croatia',
    'TN': 'Tunisia',
    'ML': 'Mali',
    'GE': 'Georgia',
    'AO': 'Angola',
    'KZ': 'Kazakhstan',
    'XE': 'Xenia',
    'ST': 'São Tomé and Príncipe',
    'GH': 'Ghana',
    'SR': 'Suriname',
    'BY': 'Belarus',
    'VG': 'British Virgin Islands',
    'SB': 'Solomon Islands',
    'BF': 'Burkina Faso',
    'XW': 'Xenomania World',
    'MA': 'Morocco',
    'CI': 'Ivory Coast',
    'SU': 'Soviet Union',
    'IR': 'Iran',
    'YU': 'Yugoslavia',
    'PY': 'Paraguay',
}




In [52]:
# Map country codes to country names
artist_10k['Country'] = artist_10k['Country'].map(country_mapping)

In [53]:
print(artist_10k['Country'].unique())

['Canada' 'United States' 'United Kingdom' 'Colombia' 'Puerto Rico' nan
 'Norway' 'France' 'Australia' 'Panama' 'Sweden' 'Germany' 'Netherlands'
 'Brazil' 'Denmark' 'Dominican Republic' 'Argentina' 'Mexico' 'Jamaica'
 'Philippines' 'South Korea' 'Spain' 'Austria' 'Italy' 'Venezuela'
 'New Zealand' 'Ireland' 'Turkey' 'Cuba' 'Belgium' 'Romania' 'Israel'
 'India' 'Poland' 'Japan' 'Uruguay' 'Finland' 'Indonesia' 'Algeria'
 'Kosovo' 'Taiwan' 'Singapore' 'Antigua and Barbuda' 'Chile' 'Portugal'
 'South Africa' 'Nigeria' 'Hong Kong' 'Aruba' 'Albania' 'Greece' 'Senegal'
 'Bosnia and Herzegovina' 'Bangladesh' 'Bolivia' 'Lithuania' 'Ukraine'
 'China' 'Russia' 'Iceland' 'Hungary' 'Pakistan' 'Cape Verde' 'Peru'
 'Malaysia' 'Honduras' 'Latvia' 'Switzerland' 'Bulgaria' 'Slovakia'
 'Democratic Republic of the Congo' 'Thailand' 'North Macedonia' 'Estonia'
 'Cameroon' 'Vietnam' 'Slovenia' 'Guatemala' 'Jordan' 'Czech Republic'
 'Moldova' 'Nicaragua' 'Croatia' 'Tunisia' 'Mali' 'Georgia' 'Angola'
 'Kazakh

In [54]:
# Preprocess the data to count the number of artists per country
artists_per_country = artist_10k['Country'].value_counts().reset_index()
artists_per_country.columns = ['Country', 'Artist Count']

# Calculate total number of artists
total_artists = artists_per_country['Artist Count'].sum()

# Calculate percentage of artists per country
artists_per_country['Percentage'] = (artists_per_country['Artist Count'] / total_artists) * 100

# Display the DataFrame with percentages
print(artists_per_country.head())



          Country  Artist Count  Percentage
0   United States          1207   19.143537
1  United Kingdom           507    8.041237
2         Germany           347    5.503569
3          Sweden           307    4.869151
4          France           259    4.107851


In [55]:
# Create a choropleth map using Plotly
fig = px.choropleth(artists_per_country,
                    locations='Country',
                    locationmode='country names',  # This tells Plotly to recognize the countries by their names
                    color='Percentage',
                    hover_name='Country',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title='Percentage of Artists per Country')
fig.update_layout(
    # height=400,
    title_font=dict(size=25),
    height=600,
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)
fig.show()

In [56]:
# Group the DataFrame by 'Country' and apply a function to get the top 5 artists based on followers
top_artists_per_country = artist_10k.groupby('Country').apply(lambda x: x.nlargest(5, 'Popularity')).reset_index(drop=True)


C:\Users\sushi\AppData\Local\Temp\ipykernel_20976\3771097867.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [57]:
top_artists_per_country[top_artists_per_country['Country'] == 'United States']

,Name,Gender,Age,Country,Genres,Popularity,Followers
347,Taylor Swift,female,30,United States,['pop'],100,95859165
348,Travis Scott,male,28,United States,"['hip hop', 'rap', 'slap house']",90,25672786
349,Lana Del Rey,female,35,United States,"['art pop', 'pop']",89,30603103
350,Ariana Grande,female,27,United States,['pop'],88,95710972
351,21 Savage,male,27,United States,"['atl hip hop', 'hip hop', 'rap']",88,15909838


In [58]:
top_artists_per_country['Top Artists'] = top_artists_per_country.groupby('Country')['Name'].transform(lambda x: ', '.join(x))



In [59]:
artist_10k[artist_10k['Country'] == 'India']

,Name,Gender,Age,Country,Genres,Popularity,Followers
484,Arijit Singh,male,33,India,"['filmi', 'modern bollywood']",88,97604060
768,Tanishk Bagchi,male,0,India,"['desi pop', 'filmi', 'modern bollywood']",75,3634127
789,Neha Kakkar,female,0,India,"['filmi', 'modern bollywood']",72,44257694
1105,Dhvani Bhanushali,female,0,India,"['desi pop', 'filmi', 'modern bollywood']",65,4636253
1267,Asees Kaur,female,0,India,"['desi pop', 'filmi', 'modern bollywood']",69,2330556
...,...,...,...,...,...,...,...
8855,Keerthana Sharma,female,0,India,['tollywood'],50,62948
8888,Nimrat Khaira,male,0,India,['punjabi pop'],56,2077522
9029,Dhanush,male,36,India,"['filmi', 'kollywood']",67,8858822
9066,Haricharan,male,33,India,['filmi'],67,2646158


In [60]:
artist_10k.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9488 entries, 0 to 9487
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        9488 non-null   object
 1   Gender      7829 non-null   object
 2   Age         9488 non-null   int64 
 3   Country     6305 non-null   object
 4   Genres      9488 non-null   object
 5   Popularity  9488 non-null   int64 
 6   Followers   9488 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 519.0+ KB


In [61]:
artist_10k['Genres'] = artist_10k['Genres'].apply(ast.literal_eval)


In [62]:
artist_10k = artist_10k[artist_10k['Genres'].map(len) > 0]

In [63]:
artist_10k.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7849 entries, 0 to 9486
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        7849 non-null   object
 1   Gender      6704 non-null   object
 2   Age         7849 non-null   int64 
 3   Country     5474 non-null   object
 4   Genres      7849 non-null   object
 5   Popularity  7849 non-null   int64 
 6   Followers   7849 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 490.6+ KB


In [64]:
country_genres = artist_10k.groupby('Country')['Genres'].sum()

In [65]:
country_genres.info()


<class 'pandas.core.series.Series'>
Index: 99 entries, Albania to Yugoslavia
Series name: Genres
Non-Null Count  Dtype 
--------------  ----- 
99 non-null     object
dtypes: object(1)
memory usage: 1.5+ KB


In [66]:
top_artists_per_country.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         373 non-null    object
 1   Gender       367 non-null    object
 2   Age          373 non-null    int64 
 3   Country      373 non-null    object
 4   Genres       373 non-null    object
 5   Popularity   373 non-null    int64 
 6   Followers    373 non-null    int64 
 7   Top Artists  373 non-null    object
dtypes: int64(3), object(5)
memory usage: 23.4+ KB


In [67]:
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in top_artists_per_country['Country'].unique()],
        value='Albania',  # Default value
        clearable=False
    ),
    dcc.Graph(id='pie-chart'),
    dash_table.DataTable(
        id='artist-table',
        columns=[
            {'name': 'Name', 'id': 'Name'},
            {'name': 'Gender', 'id': 'Gender'},
            {'name': 'Age', 'id': 'Age'},
            {'name': 'Genres', 'id': 'Genres'},
            {'name': 'Popularity', 'id': 'Popularity'},
            {'name': 'Followers', 'id': 'Followers'}
        ],
        data=[],  # Empty at initialization
        style_table={'height': '300px', 'overflowY': 'auto'}
    )
])

# Define callback to update the pie chart and the artist table based on selected country
@app.callback(
    [Output('pie-chart', 'figure'),
     Output('artist-table', 'data')],
    [Input('country-dropdown', 'value')]
)
def update_visuals(selected_country):
    # Update the pie chart
    genre_count = Counter(country_genres[selected_country])
    df_genres = pd.DataFrame(list(genre_count.items()), columns=['Genre', 'Count'])
    df_genres = df_genres.sort_values('Count', ascending=False)
    df_genres = df_genres.head(7)
    pie_chart = go.Figure(data=[go.Pie(labels=df_genres['Genre'], values=df_genres['Count'])])
    pie_chart.update_layout(title=f'Popular Genres in {selected_country}')

    # Update the artist table
    df_artists = top_artists_per_country[top_artists_per_country['Country'] == selected_country]
    artist_data = df_artists.to_dict('records')

    return pie_chart, artist_data


In [68]:

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



In [279]:
# Copyright 2024 satyabn20
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

